# Prepro proyecto Taxicab NYC

En este notebook se puede ver el preprocesamiento de los datos para encarar el proyecto del análisis de la data de los Taxis amarillos en NYC para el año 2016.

Los datos presentan una característica particular:
* En el 1er semestre encontramos columnas exactas con la posición __XY__ de inicio del viaje y, con el mismo detalle, encontramos los puntos de fin de viaje.
* En el 2do semestre desaparecen los datos de localización exacta y son reemplazados por la __ZONA__ en donde comienza y termina el viaje

Para atacar este problema, el equipo decidió hacer un mapeo de zonas desde el 2do semestre al 1ro. Esto se hizo mediante el uso de una tabla de linkeo...

In [4]:
#Librerías para poder hacer el join por ubicación geográfica
!pip install fiona shapely pyproj rtree geopandas

In [5]:
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas
import random
import numpy as np
import gc
from tqdm import tqdm
import pickle
random.seed(10)

In [6]:
#Permitimos que colab ingrese a nuestro drive y lea los archivos de la carpeta
#del TP
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd drive/MyDrive/TP_Python

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1Yh8WOgmo6rJnUJg75Hbdt3dbJ0Ilg1CX/TP_Python


---------------------------------------

## Auxiliar de downcast

El dataset es muy pesado para trabajarlo en memoria, vamos a intentar disminuírle el tamaño downcasteando los tipos de datos y generando IDs que reemplazen los datos de tipo string (generando también dics de mapeo para poder decodificarlos si es necesario)

In [7]:
# Downcast de datos del 1er semestre
downcast_1st_semester = {'VendorID': np.int8,
                         'passenger_count': np.int8,
                         'trip_duration': np.int32,
                         'pickup_longitude': np.float16,
                         'pickup_latitude': np.float16,
                         'dropoff_longitude': np.float16,
                         'dropoff_latitude': np.float16,
                         'store_and_fwd_flag': 'category',
                         'payment_type': np.int8,
                         'trip_distance': np.float32,
                         'RatecodeID': np.int8,
                         'fare_amount':np.float16,
                          'extra':np.float16,
                          'mta_tax':np.float16,
                          'tip_amount':np.float16,
                          'tolls_amount':np.float16,
                          'improvement_surcharge':np.float16,
                          'total_amount':np.float16
                          }

# Downcast de datos del 2do semestre
downcast_2nd_semester = {'VendorID': np.int8,
                         'passenger_count': np.int8,
                         'trip_duration': np.int32,
                         'PULocationID': np.str,
                         'DOLocationID': np.str,
                         'store_and_fwd_flag': 'category',
                         'payment_type': np.int8,
                         'trip_distance': np.float32,
                         'RatecodeID': np.int8,
                         'fare_amount':np.float16,
                          'extra':np.float16,
                          'mta_tax':np.float16,
                          'tip_amount':np.float16,
                          'tolls_amount':np.float16,
                          'improvement_surcharge':np.float16,
                          'total_amount':np.float16
                          }

#Las fechas las vamos a cargar con formato datetime ni bien levantamos el DF,
#ya que este tipo de dato ocupa menos lugar que el tipo de dato default de
#pandas ('O')
date = ['tpep_pickup_datetime','tpep_dropoff_datetime']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


--------------------------------------------------------------

## Lectura de DataFrames


Recolectamos la data de los diferentes meses del año. Nos quedamos con ~50K instancias por mes, para tener un total de 1.2M de intancias

In [8]:
#Trabajamos la tabla que usaremos para mapear zonas
geo_link = geopandas.read_file('taxi_zones.csv')
geo_link['the_geom'] = geopandas.GeoSeries.from_wkt(geo_link['the_geom'])
geo_link.geometry = geo_link['the_geom']

In [9]:
df = pd.DataFrame()

In [10]:
for mes in tqdm(range(1,13)):
    archivo_a_levantar = f'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-{mes:02}.csv'
    p = 0.01  #Nos quedamos con el .5% de las lineas de cada mes
    # Si el intervalo de random, que pertenece a [0,1], es mayor a 0.005 la row va a ser skippeada

    #Conectamos nuestra muestra con el dataframe que tiene las zonas geográficas
    if mes < 7:
      df_ = pd.read_csv(
              archivo_a_levantar,
              header=0, 
              index_col=False,
              skiprows=lambda i: i>0 and random.random() > p,
              dtype=downcast_1st_semester,
              parse_dates=date)

      tempPU = geopandas.GeoDataFrame(df_, geometry=geopandas.points_from_xy(df_.pickup_longitude, df_.pickup_latitude))
      tempPU = geopandas.sjoin(tempPU, geo_link[['LocationID','geometry','borough','zone']])
      tempPU.drop(['index_right','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)
      tempPU.rename(columns={'geometry':'PUgeometry','borough':'PUborough','zone':'PUzone'}, inplace=True)

      tempDO = geopandas.GeoDataFrame(df_, geometry=geopandas.points_from_xy(df_.dropoff_longitude, df_.dropoff_latitude))
      tempDO = geopandas.sjoin(tempDO, geo_link[['LocationID','geometry','borough','zone']])
      tempDO.rename(columns={'geometry':'DOgeometry','borough':'DOborough','zone':'DOzone'}, inplace=True)

      #Descartamos las instancias que tengan data de PU pero no de DO y viceversa
      df_ = tempPU.merge(tempDO[['DOgeometry','DOborough','DOzone']], left_index=True, right_index=True, how='left')
      df_ = df_.dropna()
      df_ = df_[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
                  'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
                  'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                  'tolls_amount', 'improvement_surcharge', 'total_amount', 'PUgeometry',
                  'PUborough', 'PUzone', 'DOgeometry', 'DOborough',
                  'DOzone']]
      #Borramos los temporales de la RAM
      del tempPU, tempDO
      gc.collect();

    else:
      df_ = pd.read_csv(
              archivo_a_levantar,
              header=0, 
              index_col=False,
              skiprows=lambda i: i>0 and random.random() > p,
              dtype=downcast_2nd_semester,
              parse_dates=date)

      #Merge con el dataset de zonas
      df_ = df_.merge(geo_link[['LocationID','geometry','borough','zone']], left_on='PULocationID', right_on='LocationID')
      df_.drop(['LocationID', 'PULocationID'],axis=1,inplace=True)
      df_.rename(columns={'geometry':'PUgeometry','borough':'PUborough','zone':'PUzone'}, inplace=True)

      df_ = df_.merge(geo_link[['LocationID','geometry','borough','zone']], left_on='DOLocationID', right_on='LocationID')
      df_.drop(['LocationID', 'DOLocationID'],axis=1,inplace=True)
      df_.rename(columns={'geometry':'DOgeometry','borough':'DOborough','zone':'DOzone'}, inplace=True)
      df_ = df_[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
                  'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
                  'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                  'tolls_amount', 'improvement_surcharge', 'total_amount', 'PUgeometry',
                  'PUborough', 'PUzone', 'DOgeometry', 'DOborough',
                  'DOzone']]
    #Populamos el df final
    df = pd.concat([df,df_])

    #Borramos el archivo temporal de la RAM
    del df_
    gc.collect();

 50%|█████     | 6/12 [10:29<10:48, 108.15s/it]/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)
 58%|█████▊    | 7/12 [11:10<07:10, 86.05s/it] /usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)
 67%|██████▋   | 8/12 [11:53<04:49, 72.35s/it]/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)
 75%|███████▌  | 9/12 [12:39<03:12, 64.08s/it]/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of da

In [11]:
df.reset_index(inplace=True,drop=True)

In [12]:
df.shape

(1040822, 21)

In [13]:
#Controlamos que no haya nulos
df.isna().sum().sum()

0

In [14]:
#Registros por mes
df.groupby(df.tpep_pickup_datetime.astype('datetime64').dt.month).size()

tpep_pickup_datetime
1      67798
2      71732
3      77194
4      74803
5      73317
6      69949
7     101157
8      97848
9      99739
10    106786
11     98964
12    101535
dtype: int64

In [15]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,PUgeometry,PUborough,PUzone,DOgeometry,DOborough,DOzone
0,2,2016-01-01 00:00:12,2016-01-01 00:01:17,1,0.13,1,N,2,3.0,0.5,...,0.000000,0.0,0.300049,4.300781,POINT (-74.00000 40.75000),Manhattan,East Chelsea,POINT (-74.00000 40.75000),Manhattan,East Chelsea
1,1,2016-01-29 09:19:11,2016-01-29 09:23:09,1,0.50,1,N,1,4.5,0.0,...,1.549805,0.0,0.300049,6.851562,POINT (-74.00000 40.75000),Manhattan,East Chelsea,POINT (-74.00000 40.75000),Manhattan,East Chelsea


-----------------------------------------------------

## Reducción de espacio en RAM

In [16]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040822 entries, 0 to 1040821
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1040822 non-null  int8          
 1   tpep_pickup_datetime   1040822 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  1040822 non-null  datetime64[ns]
 3   passenger_count        1040822 non-null  int8          
 4   trip_distance          1040822 non-null  float32       
 5   RatecodeID             1040822 non-null  int8          
 6   store_and_fwd_flag     1040822 non-null  category      
 7   payment_type           1040822 non-null  int8          
 8   fare_amount            1040822 non-null  float16       
 9   extra                  1040822 non-null  float16       
 10  mta_tax                1040822 non-null  float16       
 11  tip_amount             1040822 non-null  float16       
 12  tolls_amount           10408

In [17]:
#Vemos el uso de memoria por columna. Las columas de zona y borough son las más
#pesadas porque son strings, las vamos a encodear en IDs
df.memory_usage(deep=True).sort_values()

Index                         128
VendorID                  1040822
passenger_count           1040822
RatecodeID                1040822
payment_type              1040822
store_and_fwd_flag        1041046
total_amount              2081644
improvement_surcharge     2081644
tolls_amount              2081644
tip_amount                2081644
mta_tax                   2081644
extra                     2081644
fare_amount               2081644
trip_distance             4163288
tpep_dropoff_datetime     8326576
tpep_pickup_datetime      8326576
PUgeometry                8326576
DOgeometry                8326576
DOborough                68360714
PUborough                68401296
PUzone                   75857492
DOzone                   75977531
dtype: int64

In [18]:
#Generamos el set que tiene todas las zonas que existen tando para PU como para
#DO
zonas = set(df.PUzone.sort_values().unique())\
                                .union(set(df.DOzone.sort_values().unique()))
#Diccionario con el que vamos a encodear el DF
aux_zonas_encoder = dict(zip(zonas,range(len(zonas))))

#Diccionario que nos vamos a guardar para, si es necesario en algún momento del
#análisis, desencodear los IDs
aux_zonas_decoder = {}
for k,v in aux_zonas_encoder.items():
  aux_zonas_decoder[v]=k

#Hacemos el encodeo
df['PUzone'] = df.PUzone.map(aux_zonas_encoder).astype('int8')
df['DOzone']   = df.DOzone.map(aux_zonas_encoder).astype('int8')

In [19]:
#Lo mismo que la celda anterior pero para los borough
borough = set(df.PUborough.sort_values().unique()).union(set(df.DOborough.sort_values().unique()))
aux_borough_encoder = dict(zip(borough,range(len(borough))))

aux_borough_decoder = {}
for k,v in aux_borough_encoder.items():
  aux_borough_decoder[v]=k

df['PUborough']  = df.PUborough.map(aux_borough_encoder).astype('int8')
df['DOborough']  = df.DOborough.map(aux_borough_encoder).astype('int8')

In [20]:
#Vemos que el uso de memoria bajó de ~165MB a ~29MB
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040822 entries, 0 to 1040821
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1040822 non-null  int8          
 1   tpep_pickup_datetime   1040822 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  1040822 non-null  datetime64[ns]
 3   passenger_count        1040822 non-null  int8          
 4   trip_distance          1040822 non-null  float32       
 5   RatecodeID             1040822 non-null  int8          
 6   store_and_fwd_flag     1040822 non-null  category      
 7   payment_type           1040822 non-null  int8          
 8   fare_amount            1040822 non-null  float16       
 9   extra                  1040822 non-null  float16       
 10  mta_tax                1040822 non-null  float16       
 11  tip_amount             1040822 non-null  float16       
 12  tolls_amount           10408

In [21]:
#Las columnas de zone y borough ahora pesan mucho menos
df.memory_usage(deep=True).sort_values()

Index                        128
PUzone                   1040822
PUborough                1040822
DOborough                1040822
payment_type             1040822
RatecodeID               1040822
DOzone                   1040822
passenger_count          1040822
VendorID                 1040822
store_and_fwd_flag       1041046
fare_amount              2081644
mta_tax                  2081644
tip_amount               2081644
tolls_amount             2081644
improvement_surcharge    2081644
total_amount             2081644
extra                    2081644
trip_distance            4163288
tpep_dropoff_datetime    8326576
PUgeometry               8326576
tpep_pickup_datetime     8326576
DOgeometry               8326576
dtype: int64

---------------------------------------

## Save

In [22]:
#Guardamos toda la data necesaria para hacer nuestro análisis en un pickle
pickle.dump({
    'data'           :df,
    'zonas_encoder'  :aux_zonas_encoder,
    'zonas_decoder'  :aux_zonas_decoder,
    'borough_encoder':aux_borough_encoder,
    'borough_decoder':aux_borough_decoder
    }, 
    open(f'NYC_processed.pkl','wb'))